In [ ]:
!pip install pathway bokeh --quiet # This cell may take a few seconds to execute.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

#Importing and Preprocessing the Data





In [ ]:
df = pd.read_csv('/content/dataset.csv')
places = df['SystemCodeNumber'].unique()

In [ ]:
# Combine the 'LastUpdatedDate' and 'LastUpdatedTime' columns into a single datetime column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')

# Sort the DataFrame by the new 'Timestamp' column and reset the index
df = df.sort_values('Timestamp').reset_index(drop=True)

In [ ]:
# Save the selected columns to a CSV file for streaming or downstream processing
df[["SystemCodeNumber","Timestamp", "Occupancy", "Capacity"]].to_csv("parking_stream.csv", index=False)
df

In [ ]:
# Define the schema for the streaming data using Pathway
# This schema specifies the expected structure of each data row in the stream

class ParkingSchema(pw.Schema):
    SystemCodeNumber: str  # Unique identifier for the parking spot
    Timestamp: str   # Timestamp of the observation (should ideally be in ISO format)
    Occupancy: int   # Number of occupied parking spots
    Capacity: int    # Total parking capacity at the location

In [ ]:
# Load the data as a simulated stream using Pathway's replay_csv function

data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=1000)

In [ ]:
# Define the datetime format to parse the 'Timestamp' column
fmt = "%Y-%m-%d %H:%M:%S"

# Add new columns to the data stream:
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)



# Model 1- Making a simple pricing function

In [ ]:

BASE_PRICE = 10.0
ALPHA = 0.5

# Stateless price: one per timepoint
model1_output = data_with_time.with_columns(
    price=BASE_PRICE + ALPHA * (pw.this.Occupancy / pw.this.Capacity)
)




# Step 3: Visualizing Daily Price Fluctuations with a Bokeh Plot

**Note:** The Bokeh plot in the next cell will only be generated after you run the `pw.run()` cell (i.e., the final cell).


In [ ]:
import panel as pn
import bokeh.plotting
from bokeh.models import ColumnDataSource, HoverTool

pn.extension()

def make_plot_for_place(place):
    def price_plotter(source):
        fig = bokeh.plotting.figure(
            height=400,
            width=800,
            title=f"Pathway: Daily Parking Price – {place}",
            x_axis_type="datetime",
        )
        fig.add_tools(HoverTool(
            tooltips=[
                ("Time", "@t{%F %T}"),
                ("Price", "@price"),
            ],
            formatters={'@t': 'datetime'},
            mode='vline'
        ))
        fig.line("t", "price", source=source, line_width=2, color="navy")
        fig.scatter("t", "price", source=source, size=6, color="red")
        return fig

    # Filter the Pathway stream for the current place
    filtered = model1_output.filter(pw.this.SystemCodeNumber == place)
    return filtered.plot(price_plotter, sorting_col="t")
# Generate a list of plots for each place
plots = [make_plot_for_place(place) for place in places]

# Display them stacked vertically in a live-updating Panel dashboard
pn.Column(*plots).servable()


In [ ]:
# Start the Pathway pipeline execution in the background
# - This triggers the real-time data stream processing defined above
# - %%capture --no-display suppresses output in the notebook interface

%%capture --no-display
pw.run()

# Model 2


In [ ]:
# Define schema for Pathway streaming

df = pd.read_csv("/content/dataset.csv")

# Combine date and time into a single timestamp
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format='%d-%m-%Y %H:%M:%S')

# Sort by time for streaming simulation
df = df.sort_values('Timestamp')

# Add/ensure all required columns exist
df['QueueLength'] = df.get('QueueLength', 0.0)
df['TrafficLevel'] = df.get('TrafficLevel', 0.5)
df['IsSpecialDay'] = df.get('IsSpecialDay', False)
df['VehicleType'] = df.get('VehicleType', 'CAR')

# Save for streaming input
df[['Timestamp', 'SystemCodeNumber', 'Occupancy', 'Capacity',
    'QueueLength', 'TrafficLevel', 'IsSpecialDay', 'VehicleType']].to_csv("parking_stream2.csv", index=False)


class PricingSchema2(pw.Schema):
    Timestamp: str
    SystemCodeNumber: str
    Occupancy: int
    Capacity: int
    QueueLength: float
    TrafficLevel: float
    IsSpecialDay: bool
    VehicleType: str


# Load data as a stream
data = pw.demo.replay_csv("parking_stream2.csv", schema=PricingSchema2, input_rate=1000)

# Add parsed datetime columns for easier processing
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime("%Y-%m-%d %H:%M:%S"),
    day = data.Timestamp.dt.strptime("%Y-%m-%d %H:%M:%S").dt.strftime("%Y-%m-%dT00:00:00")
)


In [ ]:
model2_state = {}
vehicle_weights = {"CAR": 1.0, "BIKE": 0.7, "BUS": 1.5, "TRUCK": 1.8, "OTHER": 1.0}

def demand_based_model(occ, cap, sys, ts, q, traffic, special, vtype):
    """
    Calculates price based on a weighted demand function.
    Clamps price between 0.5x and 2x base price.
    """
    global model2_state
    day = ts.date()
    if sys not in model2_state:
        model2_state[sys] = {}
    if day not in model2_state[sys]:
        model2_state[sys][day] = 10.0  # Daily reset base price
    base = model2_state[sys][day]
    occ_rate = occ / cap
    special = 1 if special else 0
    vw = vehicle_weights.get(vtype.upper(), 1.0)
    α, β, γ, δ, ε = 1.0, 0.5, 1.0, 0.3, 0.8
    demand = α*occ_rate + β*q - γ*traffic + δ*special + ε*vw
    norm = np.clip((demand - 0.5) / 3, 0, 1)
    λ = 0.8
    price = base * (1 + λ * norm)
    return round(np.clip(price, 5.0, 20.0), 2)

# Apply Model 2 to the data stream
model2_output = data_with_time.with_columns(
    price=pw.apply(demand_based_model,
                   pw.this.Occupancy,
                   pw.this.Capacity,
                   pw.this.SystemCodeNumber,
                   pw.this.t,
                   pw.this.QueueLength,
                   pw.this.TrafficLevel,
                   pw.this.IsSpecialDay,
                   pw.this.VehicleType),
)

In [ ]:
import panel as pn
import bokeh.plotting
from bokeh.models import ColumnDataSource, HoverTool

pn.extension()

def make_plot_for_place_model2(place):
    def price_plotter(source):
        fig = bokeh.plotting.figure(
            height=400,
            width=800,
            title=f"Pathway: Demand-Based Price – {place}",
            x_axis_type="datetime",
        )
        fig.add_tools(HoverTool(
            tooltips=[
                ("Time", "@t{%F %T}"),
                ("Price", "@price"),
            ],
            formatters={'@t': 'datetime'},
            mode='vline'
        ))
        fig.line("t", "price", source=source, line_width=2, color="navy")
        fig.scatter("t", "price", source=source, size=6, color="red")
        return fig

    # Filter the Pathway stream for the current place
    filtered = model2_output.filter(pw.this.SystemCodeNumber == place)
    return filtered.plot(price_plotter, sorting_col="t")

# Generate a list of plots for each place
plots = [make_plot_for_place_model2(place) for place in places]

# Display them stacked vertically in a live-updating Panel dashboard
pn.Column(*plots).servable()


In [ ]:
# Start the Pathway pipeline execution in the background
# - This triggers the real-time data stream processing defined above
# - %%capture --no-display suppresses output in the notebook interface

%%capture --no-display
pw.run()